In [ ]:
import pandas as pd
pd.set_option("display.max_columns", None)
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import ee
import geemap
ee.Authenticate()

# Write your project ID here, in quotes
ee.Initialize(project = "anr-41793")

In [ ]:
mosquito = gpd.read_file('https://github.com/geo-di-lab/emerge-lessons/raw/refs/heads/main/docs/data/globe_mosquito.zip')

In [ ]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.
  Args:
      image (ee.Image): A Sentinel-2 image.
  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

def get_data_for_point(point):
    # Extract the point's date property
    point_date = ee.Date(point.get('MeasuredDate'))
    day_before = point_date.advance(-1, 'day')
    month_before = point_date.advance(-30, 'day')

    #### Land Surface Temperature ####
    lst = (
        ee.ImageCollection('MODIS/061/MOD11A1')
        .filterDate(day_before, point_date)
        .select(['LST_Day_1km', 'QC_Day', 'LST_Night_1km', 'QC_Night', 'Clear_day_cov', 'Clear_night_cov'])
        .median()
        # Sample LST at the point’s location
        .reduceRegion(
            reducer=ee.Reducer.median(),
            geometry=point.geometry(),
            scale=1000
        )
    )
    point = point.set(lst)

    #### Sentinel-2, Median of Previous Month (used to calculate NDVI and NDWI) ####
    sentinel2 = (
        ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterDate(month_before, point_date)
        .map(mask_s2_clouds)
        .median()
        # Sample median band values at the point’s location
        .reduceRegion(
            reducer=ee.Reducer.median(),
            geometry=point.geometry(),
            scale=10
        )
    )
    point = point.set(sentinel2)

    #### Precipitation ####
    rain = (
        ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
        .filterDate(day_before, point_date)
        .select('precipitation')
        .sum()
        # Sample median band values at the point’s location
        .reduceRegion(
            reducer=ee.Reducer.median(),
            geometry=point.geometry(),
            scale=5566
        )
    )
    point = point.set(rain)

    return point

In [ ]:
mosquito_2024 = mosquito[mosquito['MeasuredDate'].str.startswith('2024')]

In [ ]:
# Apply the function to all points in 2024
points_fc = geemap.geopandas_to_ee(mosquito_2024)
result_fc = points_fc.map(get_data_for_point)

In [ ]:
data = geemap.ee_to_gdf(result_fc)

## Calculate NDVI
data['NDVI'] = (data['B8'] - data['B4']) / (data['B8'] + data['B4'])

## Calculate NDWI
data['NDWI'] = (data['B3'] - data['B8']) / (data['B3'] + data['B8'])

In [ ]:
data

In [ ]:
data = gpd.read_file('https://github.com/geo-di-lab/emerge-lessons/raw/refs/heads/main/docs/data/globe_mosquito_env_2024.zip')

In [ ]:
# Remove outliers using the IQR method

larvae_count = data[data['LarvaeCountProcessed'] > 0]

Q1 = larvae_count['LarvaeCountProcessed'].quantile(0.25)
Q3 = larvae_count['LarvaeCountProcessed'].quantile(0.75)
IQR = Q3 - Q1

# identify outliers
threshold = 1.5
data_cleaned = data[(data['LarvaeCountProcessed'] >= Q1 - threshold * IQR) & (data['LarvaeCountProcessed'] <= Q3 + threshold * IQR)]

In [ ]:
fig, (ax1, ax2) = plt.subplots(2)

ax1.hist(data_cleaned[data_cleaned['LarvaeCountProcessed'] > 0]['NDVI'], bins=20)
ax1.set_title("With Larvae")

ax2.hist(data_cleaned[data_cleaned['LarvaeCountProcessed'] == 0]['NDVI'], bins=20)
ax2.set_title("Without Larvae")

plt.suptitle("Distribution of Vegetation (NDVI) at Mosquito Observations")
plt.tight_layout()

In [ ]:
fig, (ax1, ax2) = plt.subplots(2)

ax1.hist(data_cleaned[data_cleaned['LarvaeCountProcessed'] > 0]['precipitation'], bins=200)
ax1.set_title("With Larvae")
ax1.set_xlim([1, 20])
ax1.set_ylim([0, 50])

ax2.hist(data_cleaned[data_cleaned['LarvaeCountProcessed'] == 0]['precipitation'], bins=200)
ax2.set_title("Without Larvae")
ax2.set_xlim([1, 20])
ax2.set_ylim([0, 50])

plt.suptitle("Distribution of Precipitation (mm/day) at Mosquito Observations")
plt.tight_layout()

In [ ]:
from scipy.stats import spearmanr

In [ ]:
cols = ['LarvaeCountProcessed', 'precipitation', 'LST_Day_1km', 'LST_Night_1km', 'NDVI', 'NDWI', 'MeasurementLongitude', 'MeasurementLatitude']
sns.heatmap(data_cleaned[cols].corr(method='spearman'), annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)

In [ ]:
def interpret_spearmanr(variable, alpha=0.05):
    """Interprets the statistical significance of a Spearman's correlation result"""
    correlation_coefficient, p_value = spearmanr(data_cleaned[variable], data_cleaned['LarvaeCountProcessed'], nan_policy='omit')

    print(f"Spearman's correlation coefficient: {correlation_coefficient:.3f}")
    print(f"P-value: {p_value}")

    if p_value < alpha:
        direction = "positive" if correlation_coefficient > 0 else "negative"
        print(f"The result is statistically significant, with a {direction} correlation.")
    else:
        print("The result is not statistically significant.")

In [ ]:
print(f"Number of observations: {len(data_cleaned)}")

print("\n---NDVI and Mosquito Larvae Count---")
interpret_spearmanr('NDVI')

print("\n---Precipitation and Mosquito Larvae Count---")
interpret_spearmanr('precipitation')

print("\n---Night Temperature and Mosquito Larvae Count---")
interpret_spearmanr('LST_Night_1km')

print("\n---Day Temperature and Mosquito Larvae Count---")
interpret_spearmanr('LST_Day_1km')

print("\n---NDVI and Mosquito Larvae Count---")
interpret_spearmanr('NDVI')

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 4))

sns.regplot(x = "NDVI", y = "LarvaeCountProcessed", data = data_cleaned[data_cleaned['LarvaeCountProcessed'] > 0],
           line_kws = {"color": "darkblue"}, ax = ax1)
ax1.set_title('Vegetation (NDVI)')

sns.regplot(x = "precipitation", y = "LarvaeCountProcessed", data = data_cleaned[data_cleaned['LarvaeCountProcessed'] > 0],
           line_kws = {"color": "darkblue"}, ax = ax2)
ax2.set_title('Precipitation')

sns.regplot(x = "LST_Night_1km", y = "LarvaeCountProcessed", data = data_cleaned[data_cleaned['LarvaeCountProcessed'] > 0],
           line_kws = {"color": "darkblue"}, ax = ax3)
ax3.set_title('Nighttime Land Surface Temperature')

plt.suptitle('Environmental Conditions at Mosquito Observations')
plt.tight_layout()
plt.show()